In [4]:
import tensorflow
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [5]:
#load the trained model and pickle files
model = load_model('final_model_ann.keras')

with open('label_enc_gen.pkl','rb') as file:
    label_encode = pickle.load(file)

with open('ohe_geo.pkl','rb') as file:
    ohe_geo_encode = pickle.load(file)

with open('sc.pkl','rb') as file:
    sc_data = pickle.load(file)


In [39]:
input_data = {
    'CreditScore' : 719,
    'Geography' : 'France',
    'Gender' : 'Male',
    'Age' : 43,
    'Tenure' : 3,
    'Balance' : 67000,
    'NumOfProducts' : 1,
    'HasCrCard' : 1,
    'IsActiveMember' : 1,
    'EstimatedSalary' : 55000
}

In [40]:
geo_encoded = ohe_geo_encode.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=ohe_geo_encode.get_feature_names_out(['Geography']))
geo_encoded_df.head()

d:\study material\My Projects\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [41]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,719,France,Male,43,3,67000,1,1,1,55000


In [42]:
input_df['Gender'] = label_encode.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,719,France,1,43,3,67000,1,1,1,55000


In [43]:
#concat one hot encoded data
input_df = pd.concat([input_df.drop("Geography",axis=1), geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,719,1,43,3,67000,1,1,1,55000,1.0,0.0,0.0


In [44]:
#Scaling the input Data
input_scaled = sc_data.transform(input_df)
input_scaled

array([[ 0.69896483,  0.91324755,  0.3900109 , -0.69539349, -0.14573426,
        -0.91668767,  0.64920267,  0.97481699, -0.78989886,  1.00150113,
        -0.57946723, -0.57638802]])

In [46]:
prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 13ms/step


array([[0.0540896]], dtype=float32)

In [47]:
pred_prob = prediction[0][0]
pred_prob

0.0540896

In [48]:
if pred_prob > 0.5:
    print('The Customer is likely to Churn')
else :
    print('The Customer is unlikely to Churn')

The Customer is unlikely to Churn
